In [52]:
import cv2
import numpy as np
import collections
    
def TwoPass(name):
    img = cv2.imread(name, 0)
    # ensure binary
    img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]
    
    num_part = 0
    labeled_img = []
    h, w = img.shape
    labels = [[0 for i in range(w)] for j in range(h)]
    
    # A table for holding all pixel of img
    W_img = collections.defaultdict(int)

    # First pass
    for i in range(h):
        for j in range(w):
            if img[i][j] == 255:
                if (i-1) in range(h):
                    upper = labels[i-1][j]
                else:
                    upper = 0
                    
                if (j-1) in range(w):
                    left = labels[i][j-1]
                else:
                    left = 0
                # define all left and upper    
                    
                if upper == 0 and left == 0:
                    num_part += 1
                    labels[i][j] = num_part
                    
                elif left == 0 or upper == 0:
                    labels[i][j] = max(upper, left)
                    
                elif left == upper:
                    labels[i][j] = upper
                    
                else:
                    labels[i][j] = min(upper, left)
                    W_img[max(upper, left)] = max(W_img[max(upper, left)], labels[i][j])

    # Second pass
    def find_smallest_label(node):
        if node not in W_img:
            myset.add(node)
            return node
        else:
            return find_smallest_label(W_img[node])
    
    areas = collections.defaultdict(int)
    collected_pixel = collections.defaultdict(int)
        
    myset = set()

    for val in W_img:
        W_img[val] = find_smallest_label(W_img[val])
        


    dict = collections.defaultdict(int)
    
    for index, num in enumerate(myset):
        dict[num] = index + 1

    for i in range(h):
        for j in range(w):
            if img[i][j] == 255 and labels[i][j] in W_img:
                labels[i][j] = W_img[labels[i][j]]
                collected_pixel[labels[i][j]] += 1
            else:
                collected_pixel[labels[i][j]] += 1
                
    ###########################   
    def filter():
        # this area can change to fitted one
        min_area = 2000
        for key in areas:
            if areas[key] < min_area:
                areas[key] = -1
            
    ############################
    labeled_img = img
    for i in range(h):
        for j in range(w):
            filter()
            if labels[i][j]:
                if areas[labels[i][j]] == -1:
                    color = 0
                else:
                    color = 255/(len(myset)+1) * dict[labels[i][j]]
                labeled_img[i][j] = color
    ###########################
#     print(img)

#     label_hue = np.uint8(179 * img/np.max(img))
#     blank_ch = 255 * np.ones_like(label_hue)
#     labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

#     # cvt to BGR for display
#     labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

#     # set bg label to black
#     labeled_img[label_hue==0] = 0

    
    print('Nums:', len(myset))
#     cv2.imwrite('labeled.png', labeled_img)
    cv2.imshow('labeled.png', labeled_img)
    cv2.waitKey()


In [53]:

# TwoPass("test.bmp")
# TwoPass("face.bmp")
TwoPass("gun.bmp")
# TwoPass("face_old.bmp")

Nums: 4


In [6]:
import cv2
import numpy as np

# img = cv2.imread('test.bmp', 0)
img = cv2.imread('face.bmp', 0)
# img = cv2.imread('gun.bmp', 0)
# img = cv2.imread('face_old.bmp',0)

num_labels, labels_im = cv2.connectedComponents(img)
# num_labels, labels_im = MS(img)


def CCL(labels):
    # Map component labels to hue val
    label_hue = np.uint8(179 * labels/np.max(labels))
    blank_ch = 255 * np.ones_like(label_hue)
    labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

    # cvt to BGR for display
    labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

    # set bg label to black
    labeled_img[label_hue==0] = 0

    cv2.imshow('labeled.png', labeled_img)
    cv2.imwrite('labeled.png', labeled_img)
    cv2.waitKey()
    
print ('num:',num_labels-1)
CCL(labels_im)


num: 6


In [37]:
import cv2
import numpy as np

def zerolistmaker(n):
    listofzeros = [0] * n
    return listofzeros


img = cv2.imread('gun.bmp', 0)

img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]  # ensure binary
retval, labels = cv2.connectedComponents(img)

##################################################
# ENLARGEMENT
##################################################
sorted_labels = labels.ravel()
sorted_labels = np.sort(sorted_labels)


maxPixel = 50  # eliminate elements with less than maxPixel

# detect how often an element occurs
i=0
counter=0
counterlist = zerolistmaker(retval)

while i < len(sorted_labels):
    if sorted_labels[i] == counter:
        counterlist[counter] = counterlist[counter] + 1
    else:
        counter = counter + 1
        i = i - 1

    i = i + 1


# delete small pixel values
i=0
while i < len(counterlist):
    if counterlist[i] < maxPixel:
        counterlist[i] = 0
    i = i + 1

i=0
counterlisthelper = []
while i < len(counterlist):
    if counterlist[i] == 0:
        counterlisthelper.append(i)
    i = i + 1

i=0
j=0
k=0
while k < len(counterlisthelper):
    while i < labels.shape[0]:
        while j < labels.shape[1]:
            if labels[i,j] == counterlisthelper[k]:
                labels[i,j] = 0
            else:
                labels[i,j] = labels[i,j]
            j = j + 1
        j = 0
        i = i + 1
    i = 0
    j = 0
    k = k + 1

##################################################
##################################################

# Map component labels to hue val
label_hue = np.uint8(179*labels/np.max(labels))
blank_ch = 255*np.ones_like(label_hue)
labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])

# cvt to BGR for display
labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)

# set bg label to black
labeled_img[label_hue==0] = 0

cv2.imshow('labeled.png', labeled_img)
cv2.waitKey()

-1

In [55]:
import cv2 as cv
import collections

def CCL(path):
    # read image
    img = cv.imread(path, cv.IMREAD_GRAYSCALE)

    # get image height and width
    height, width = img.shape

    # initialize variables
    num = 0
    label_img = []
    label = [[0 for i in range(width)] for j in range(height)]
    E_table = collections.defaultdict(int)

    # first pass
    for i in range(height):
        for j in range(width):
            # if pixel is white
            if img[i,j] == 255:
                # calculate labels of upper and left pixel w/ boundary conditions
                if i-1 in range(height):
                    upper = label[i-1][j]
                else:
                    upper = 0
                if j-1 in range(width):
                    left = label[i][j-1]
                else:
                    left = 0

                if upper == 0 and left == 0:
                    num += 1
                    label[i][j] = num
                elif left == 0 or upper == 0:
                    label[i][j] = max(upper, left)
                elif left == upper:
                    label[i][j] = upper
                else:
                    label[i][j] = min(upper, left)
                    E_table[max(upper, left)] = max(E_table[max(upper, left)], label[i][j])

    # second pass
    # set for holding label roots
    s = set()

    # helper function find_root -> this is basically union find
    def find_root(node):
        if node not in E_table:
            s.add(node)
            return node
        else:
            return find_root(E_table[node])

    # rearrange E_table to have all labels point to their roots
    for val in E_table:
        E_table[val] = find_root(E_table[val])

    # create true labels for each unique label that we've acquired
    ## size filter: for gun.bmp only ##
    areas = collections.defaultdict(int)
    ###                             ###

    dict = collections.defaultdict(int)
    for ind, num in enumerate(s):
        dict[num] = ind+1

    for i in range(height):
        for j in range(width):
            # if pixel is white
            # check E_table to see if it should be replaced
            if img[i,j] == 255 and label[i][j] in E_table:
                label[i][j] = E_table[label[i][j]]
                ## for size filter ##
                areas[label[i][j]] += 1
            else:
                areas[label[i][j]] += 1
                ###               ###

    ## for size filter ##
#     min_area = 1000
#     for key in areas:
#         if areas[key] < min_area:
#             areas[key] = -1
    ####

    # create label_img
    label_img = img
    for i in range(height):
        for j in range(width):
            if label[i][j]:
                # gun.bmp ONLY -> for size filter
                if areas[label[i][j]] == -1:
                    color = 0
                else:
                    color = 255 /(len(s)+1) * dict[label[i][j]]
                label_img[i][j] = color

    # returning label_img doesn't do much, so i have the function show it as well
#     title = (path.split('/')[1])[:-4] + 'size_filtered.jpg'
    cv.imwrite('good.jpg', label_img)
    cv.imshow('good', label_img)
    cv.waitKey(0)
    print('num: ', len(s))
    return label_img, num

# CCL("test.bmp")
# CCL("face.bmp")
CCL("gun.bmp")
# CCL("face_old.bmp")

num:  4


(array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 20)